In [2]:
import sys
curr_dir ='/home/users/maali/Computer_vision_SOC'
sys.path.append(curr_dir)


import sys
import os
from ImageProcessor import ImageProcessor
from Dataset import CustomImageDataGenerator
def get_run_logdir(root_logdir):
    import time 
    run_id = time.strftime("run_%y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir,run_id)

run_logdir = get_run_logdir(os.path.join(curr_dir,"my_logs"))

In [3]:

train_image_folder = '/home/users/maali/Computer_vision_SOC/spark-2022-stream-1/train/'
val_image_folder = '/home/users/maali/Computer_vision_SOC/spark-2022-stream-1/val/'
test_image_folder = '/home/users/maali/Computer_vision_SOC/spark-2022-stream-1/test/'


train_data_csv = '/home/users/maali/Computer_vision_SOC/spark-2022-stream-1/labels/train.csv'
val_data_csv = '/home/users/maali/Computer_vision_SOC/spark-2022-stream-1/labels/val.csv'


image_processor = ImageProcessor(train_image_folder,val_image_folder,train_data_csv,val_data_csv)

train_data_pd,val_data_pd = image_processor.train_df,image_processor.val_df

#image_processor.display_random_image_per_class(train_data_pd)
#image_processor.plot_feature_distribution(train_data_pd, 'class')

In [4]:

# Set the batch size and image size
batch_size = 32
image_size = (256, 256)
sample_size= 20000
# Create the custom data generator
train_generator = CustomImageDataGenerator(train_data_pd, train_image_folder, image_size=image_size,batch_size=batch_size)
val_generator = CustomImageDataGenerator(val_data_pd, val_image_folder,image_size=image_size,batch_size=batch_size)


2023-10-17 09:14:12.344634: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX512F
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import Sequence

class TransferVGG16:
    def __init__(self, input_shape=(224, 224, 3), num_classes=11):
        self.input_shape = input_shape
        self.num_classes = num_classes
        self.strategy = tf.distribute.MirroredStrategy()  # Initialize MirroredStrategy
        self.model = self.build_model()

    def build_model(self):
        with self.strategy.scope():  # Create model within the strategy scope
            base_model = VGG16(
                include_top=False,
                weights='imagenet',
                input_shape=self.input_shape
            )
            
            for layer in base_model.layers:
                layer.trainable = False

            model = models.Sequential()
            model.add(base_model)

            model.add(Conv2D(64, (7, 7), 1, activation='relu', padding='same'))
            model.add(MaxPooling2D())

            model.add(Conv2D(128, (7, 7), 1, activation='relu', padding='same'))
            model.add(MaxPooling2D())

            model.add(Conv2D(512, (7, 7), 1, activation='relu', padding='same'))
            model.add(MaxPooling2D())

            model.add(layers.Flatten())
            model.add(layers.Dense(4096, activation='relu'))
            model.add(layers.Dense(4096, activation='relu'))
            model.add(layers.Dense(self.num_classes, activation='softmax'))

        return model

    def compile_model(self, learning_rate=0.001):
        with self.strategy.scope():
            optimizer = Adam(learning_rate=learning_rate, epsilon=0.01)
            loss_fn = tf.keras.losses.CategoricalCrossentropy()
            metrics = ['accuracy']

            self.model.compile(optimizer=optimizer, loss=loss_fn, metrics=metrics)

    def train_model(self, train_gen, val_gen, callbacks=[], epochs=10):
        with self.strategy.scope():
            self.model.fit(
                train_gen.data,
                validation_data=val_gen.data,
                callbacks=callbacks,
                epochs=epochs,
                steps_per_epoch=len(train_gen),
                validation_steps=len(val_gen)
            )

    def evaluate_model(self, test_images, test_labels):
        evaluation_metrics = self.model.evaluate(test_images, test_labels)
        return {
            'loss': evaluation_metrics[0],
            'accuracy': evaluation_metrics[1]
        }

# Create the TransferVGG16 instance
transfer_vgg16 = TransferVGG16(input_shape=(image_size[0], image_size[1], 3), num_classes=11)

# Compile the model
transfer_vgg16.compile_model()

# Define your callbacks
tensorboard_cb = tf.keras.callbacks.TensorBoard(run_logdir)
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("spacecraft_classifier", save_best_only=True)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-6)

# Train the model using the train_model method
transfer_vgg16.train_model(train_generator, val_generator, callbacks=[tensorboard_cb, checkpoint_cb, early_stopping_cb, reduce_lr], epochs=100)

# Save the model
transfer_vgg16.model.save("spacecraft_classifier")


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


2023-10-17 09:15:02.804147: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2023-10-17 09:15:02.804185: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2023-10-17 09:15:02.804229: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.
2023-10-17 09:15:02.815084: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:695] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorSliceDataset/_2"
op: "TensorSliceDataset"
input: "Placeholder/_0"
input: "Placeholder/_1"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_STRING
      type: DT_FLOAT
    }
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
      }
      shape {
        dim {
          size: 11
        }
      }
    }
  }
}

2023-10-17 09:15:02.863225: I tensorflow/compil

Epoch 1/100
   1/2062 [..............................] - ETA: 5:17:02 - loss: 2.3976 - accuracy: 0.0000e+00

2023-10-17 09:15:12.301191: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2023-10-17 09:15:12.301222: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.


   2/2062 [..............................] - ETA: 1:06:26 - loss: 2.3873 - accuracy: 0.2969    

...

2023-10-17 10:15:26.741269: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:695] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorSliceDataset/_2"
op: "TensorSliceDataset"
input: "Placeholder/_0"
input: "Placeholder/_1"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_STRING
      type: DT_FLOAT
    }
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
      }
      shape {
        dim {
          size: 11
        }
      }
    }
  }
}



2062/2062 [==============================] - 4734s 2s/step - loss: 0.3779 - accuracy: 0.8828 - val_loss: 14.7746 - val_accuracy: 0.0904


2023-10-17 10:33:57.459568: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: spacecraft_classifier/assets
Epoch 2/100
   1/2062 [..............................] - ETA: 29:46 - loss: 4.0599e-04 - accuracy: 1.0000WARNING:tensorflow:Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches (in this case, 206200 batches). You may need to use the repeat() function when building your dataset.


KeyboardInterrupt: 